In [1]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
data = pd.read_csv("spam.csv")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [2]:
data = data.drop(data.columns[[2, 4, 3]], axis=1)                 # remove the empty valued columns
data.rename(columns={'v1': 'spam', 'v2': 'email'}, inplace=True)  # rename column name v1 with spam and v2 with email
data.loc[data['spam']=='spam', 'spam'] = 1                        # replace all spam with 1
data.loc[data['spam']=='ham', 'spam'] = 0                         # replace all ham with 0
data = data[['email', 'spam']]                                    # interchange columns

data.head()

,email,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [3]:
text = input("Enter the Text: ")                                  #input text to classify spam/ham

from sklearn.model_selection import train_test_split
x = data["email"]
y = data["spam"]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33)

#variables, dictionary, list used
positiveTotal = 0
negativeTotal = 0
pA = 0
pNotA = 0
trainPositive = {}
trainNegative = {}
totalWords = []
alpha = 1

def process(text):   
    text = text.lower()                                                        # lowercase it 
    text = ''.join([t for t in text if t not in string.punctuation])           # remove punctuation  
    text = [t for t in text.split() if t not in stopwords.words('english')]    # remove stopwords  
    st = Stemmer()                                                             # stemming
    text = [st.stem(t) for t in text]   
    return text                                                                # return token list

#reading words from a specific email
def processEmail(body, label):
    global positiveTotal, negativeTotal, totalWords
    body = process (body)
    for word in body:
        if label == 1:
            trainPositive[word] = trainPositive.get(word, 0) + 1
            positiveTotal += 1
        else:
             trainNegative[word] = trainNegative.get(word, 0) + 1             
             negativeTotal += 1  
        totalWords.append(word)

def train():  
  global pA, pNotA  
  total = 0
  numSpam = 0
  for email,spam in zip(x_train,y_train):
     if spam == 1 :           
       numSpam +=1
     total += 1
     processEmail(email, spam)
     pA = numSpam/float(total)
     pNotA = 1-pA
    
train()    
     
#gives the conditional probability p(B_i/A_x)
def conditionalWord(word, spam):   
    global alpha, numWords
    if spam:
       return (trainPositive.get(word,0)+alpha)/(float)(positiveTotal+alpha*numWords)
    return (trainNegative.get(word,0)+alpha)/(float)(negativeTotal+alpha*numWords)

def conditionalEmail(body, spam) :    
  result =1.0
  for word in body:
    result *= conditionalWord(word, spam)
  return result

#classifies a new email as spam or not spam
def classify(email):
  global pA, pNotA, numWords, totalWords  
  numWords=len(set(totalWords))
  isSpam = pA * conditionalEmail(email, True)        # P(A|B)
  notSpam = pNotA * conditionalEmail(email, False)   # P(¬A|B)  
  return isSpam > notSpam
  
x=classify(text)

if(x==1):
    print ("The text entered is Spam")
else:
    print ("The text entered is Not Spam")


Enter the Text: hiii
The text entered is Spam


In [4]:
trainNegative

{'watch': 45,
 'cartoon': 3,
 'listen': 12,
 'music': 3,
 'amp': 57,
 'eve': 7,
 'go': 276,
 'templ': 3,
 'church': 2,
 'u': 657,
 'ill': 168,
 'call': 191,
 '2mrw': 4,
 'ninish': 1,
 'address': 9,
 'icki': 1,
 'american': 2,
 'freek': 1,
 'wont': 36,
 'stop': 32,
 'callin': 2,
 '2': 214,
 'bad': 23,
 'jen': 2,
 'k': 51,
 'eh': 8,
 'alright': 14,
 'make': 92,
 'sure': 58,
 'car': 31,
 'back': 93,
 'tonight': 38,
 'turn': 10,
 'heater': 2,
 'set': 17,
 'ltgt': 181,
 'degre': 2,
 'still': 94,
 'grand': 3,
 'prix': 1,
 'oh': 71,
 'brand': 2,
 '4': 111,
 'let': 60,
 'bill': 5,
 'miller': 1,
 'hey': 64,
 'mate': 10,
 'how': 28,
 'honeydid': 1,
 'ave': 4,
 'good': 148,
 'holiday': 9,
 'gimmi': 1,
 'de': 16,
 'gossx': 1,
 'sorri': 98,
 'dude': 11,
 'dont': 181,
 'know': 166,
 'forgot': 20,
 'even': 60,
 'dan': 1,
 'remind': 6,
 'hope': 80,
 'guy': 40,
 'fun': 18,
 'pen': 1,
 'thing': 80,
 'beyond': 3,
 'joke': 10,
 'biro': 1,
 'master': 3,
 'cant': 89,
 'ever': 23,
 'keep': 53,
 'safe': 9,
 '

In [ ]:
mess="hii"